In [1]:
# Importing Necessary Libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import imghdr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping

2024-02-13 22:51:10.960153: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 22:51:10.993652: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 22:51:11.191931: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 22:51:11.192010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 22:51:11.215266: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [ ]:
# Mount Drive
from google.colab import drive

drive.mount("/content/gdrive")

In [ ]:
# Define Data Paths
train_dir = os.path.join(data_path, 'train')
valid_dir = os.path.join(data_path, 'valid')
test_dir = os.path.join(data_path, 'test')

In [ ]:
# Load Data
data_train = tf.keras.utils.image_dataset_from_directory(train_dir)
data_train = data_train.map(lambda x, y: (x/255, y))

data_valid = tf.keras.utils.image_dataset_from_directory(valid_dir)
data_valid = data_valid.map(lambda x, y: (x/255, y))

data_test = tf.keras.utils.image_dataset_from_directory(test_dir)
data_test = data_test.map(lambda x, y: (x/255, y))

# Get Data Iterator
data_iterator = data_train.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
# Print Size of Dataset
print(f"No. of Train Batches: {len(data_train)}")
print(f"No. of Valid Batches: {len(data_valid)}")
print(f"No. of Test Batches: {len(data_test)}")

In [ ]:
 # Plot Example Data
  fig, axs = plt.subplots(ncols=2, figsize=(12,12))

  axs[0].imshow(batch[0][5])
  axs[0].set_title(batch[1][5])

  axs[1].imshow(batch[0][1])
  axs[1].set_title(batch[1][1])

  plt.show()

In [ ]:
# Define The Model
model = Sequential()

model.add(Conv2D(16, (3,3), 1, activation="relu", input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation="relu"))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss=tf.losses.BinaryCrossentropy(), metrics=["accuracy"])

model.summary()

In [ ]:
# Define Callback
early_stopping = EarlyStopping(monitor='val_loss',  # Metric to monitor
                               patience=15,  # Number of epochs with no improvement after which training will be stopped
                               verbose=1,
                               mode='min',  # The direction is automatically inferred if not set
                               restore_best_weights=True)  # Whether to restore model weights from the epoch with the best value of the monitored quantity

In [ ]:
# Train Model
history = model.fit(data_train,
                    epochs=100,
                    validation_data=data_valid,
                    callbacks=[early_stopping])

In [ ]:
# Plot Loss
plt.figure()
plt.plot(history.history["loss"], color="teal", label="train_loss")
plt.plot(history.history["val_loss"], color="orange", label="val_loss")
plt.suptitle("Loss")
plt.legend(loc="best")
plt.show()

In [ ]:
# Metrics
precision = Precision()
recall = Recall()
binary_accuracy = BinaryAccuracy()

for batch in data_test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    binary_accuracy.update_state(y, yhat)

print(f"Precision: {precision.result()}")
print(f"Recall: {recall.result()}")
print(f"Binary Accuracy: {binary_accuracy.result()}")

In [ ]:
# Function to Classify Individual Image
def img_classify(img_path):
    img = cv2.imread(img_path)
    img = tf.image.resize(img, (256,256))
    plt.imshow(img.numpy().astype(int))
    plt.show()

    yhat = model.predict(np.expand_dims(img/255, 0))
    if yhat > 0.5:
        print(f"It's Real: {yhat}")
    else:
        print(f"It's Fake: {yhat}")

In [ ]:
# Save the Model
model.save(os.path.join(model_path, "Real_DeepFake_Classifier.h5"))